In [1]:
import torch, pandas as pd, numpy as np, warnings
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, roc_curve, roc_auc_score, RocCurveDisplay, make_scorer
from sklearn.model_selection import train_test_split, cross_val_predict, GridSearchCV, RandomizedSearchCV
from itertools import combinations, chain
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")

In [2]:
#load the cleaned data
df = pd.read_csv('https://raw.githubusercontent.com/tanujmath/CreditGuard/main/data/cs-training-new.csv')

## We do the Model Fitting twice, one with scaling and one without

### Unscaled Model Fitting

In [3]:
#Split the data into training and testing
X = df.iloc[:,2:9]
y=df['SeriousDlqin2yrs']


X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=14,
                                   test_size=0.2,
                                   shuffle=True)

#### Linear Regression

In [4]:
# Linear Regression with standardized data

l_reg = LinearRegression(copy_X=True, fit_intercept=True)
l_reg.fit(X_train,y_train)

# Calculate the AUC
l_reg_auc = roc_auc_score(y_test, l_reg.predict(X_test))
l_reg_auc

0.6852609404024366

#### Ridge Regularization

In [5]:
reg_ridge = Ridge(alpha=0.2, fit_intercept=True, copy_X=True)
reg_ridge.fit(X_train, y_train)

# Calculate the AUC
ridge_auc = roc_auc_score(y_test, reg_ridge.predict(X_test))
ridge_auc

0.6852609130934182

#### Lasso Regularization

In [6]:
reg_lasso = Lasso(alpha=0.2, fit_intercept=True, copy_X=True)
reg_lasso.fit(X_train, y_train)

# Calculate the AUC
lasso_auc = roc_auc_score(y_test, reg_lasso.predict(X_test))
lasso_auc

0.6325474480541506

#### Logistic Regression

In [7]:
#Define the model with random hyperparameters
logit = LogisticRegression(n_jobs = -1)

#Fit data into the model
logit.fit(X_train, y_train)

#Predict probabilities
logit_scores_proba = logit.predict_proba(X_test)
preds = logit_scores_proba[:,1]  # predictions

log_auc = round(roc_auc_score(y_test,preds),5)

#Printing area under curve
print('AUC Score : ',  log_auc)

AUC Score :  0.64351


#### Hyperparameter tuning

In [8]:
# Initialize logistic regression model
model = LogisticRegression(solver='liblinear', n_jobs=-1)

# Define hyperparameters to tune
hyperparameters = {'C': np.logspace(-4, 4, 10), 'penalty': ['l1', 'l2'],
                   'max_iter': list(range(100,800,100))}

# Define scorer based on AUC
scorer = make_scorer(roc_auc_score, needs_proba=True)

## GRIDSEARCH

# Initialize GridSearchCV
gridsearch = GridSearchCV(model, hyperparameters, scoring=scorer, cv=5)

# Fit GridSearchCV on training data
gridsearch.fit(X_train, y_train)

# Get the best model
best_model = gridsearch.best_estimator_


# Predict probabilities on test data using the best model
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate AUC
log_grid_auc = round(roc_auc_score(y_test, y_pred_proba),5)

print(f'The AUC of the test set through gridsearch tuning is {log_grid_auc}')

## RANDOMSEARCH

# Initialize logistic regression model
model = LogisticRegression(solver='liblinear', n_jobs=-1)


# Initialize RandomizedSearchCV
randomsearch = RandomizedSearchCV(model, hyperparameters, scoring=scorer, cv=5)

# Fit RandomizedSearchCV on training data
randomsearch.fit(X_train, y_train)

# Get the best model
best_model = randomsearch.best_estimator_


# Predict probabilities on test data using the best model
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Calculate AUC
log_random_auc = round(roc_auc_score(y_test, y_pred_proba),5)

print(f'The AUC of the test set through randomsearch tuning is {log_random_auc}')


The AUC of the test set through gridsearch tuning is 0.65679
The AUC of the test set through randomsearch tuning is 0.65724


#### Random Forests

In [9]:
# Define the model
rf = RandomForestClassifier()

# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'n_jobs': [-1]
}

# Define scorer based on AUC
scorer = make_scorer(roc_auc_score, needs_proba=True)

# Initialize RandomizedSearchCV
randomsearch = RandomizedSearchCV(rf, param_grid, scoring=scorer, cv=5, verbose = 1)

# Fit RandomizedSearchCV on training data
randomsearch.fit(X_train, y_train)

# Get the best model
best_model = randomsearch.best_estimator_

# Get the best parameters
best_params = randomsearch.best_params_

#print(f"Best parameters: {best_params}")

# Fit the model with the best parameters
rf_best = RandomForestClassifier(**best_params)
rf_best.fit(X_train, y_train)

# Make predictions
y_pred = rf_best.predict(X_test)
y_pred_proba = rf_best.predict_proba(X_test)[:, 1]

# Evaluate the model
rf_auc = round(roc_auc_score(y_test, y_pred_proba),5)

print(f"AUC: {rf_auc}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
AUC: 0.81529


#### XGBoost

In [10]:
# Define the model. If GPU is present, use GPU.
model = XGBClassifier(tree_method='hist', device = 'cuda')

# Define the parameters for the RandomizedSearch
params = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}

#Initialize randomized search
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=5,
                                   scoring='roc_auc', n_jobs=-1, cv=5, verbose=1)

# Fit the model to the training data
random_search.fit(X_train, y_train)

# Make predictions
y_pred = random_search.predict(X_test)
y_pred_proba = random_search.predict_proba(X_test)[:, 1]

# Evaluate the model
xg_auc = roc_auc_score(y_test, y_pred_proba)

print(f"AUC: {xg_auc}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
AUC: 0.8164034486920891


### Now we use scaled data

In [11]:
# first make a deep copy of the data
df2=df.copy(deep=True)

# Make a scaler object : i.e. the StandardScaler object below which will know what to do make the standardization in the way we want
scaler=StandardScaler()

# fit the scaler : calling 'fit' means, the case of standardizing here, knowing the columns and the mean, standard deviation of the each of the columns
scaler.fit(df2.iloc[:,2:9])

# scaling the data
df_scale =scaler.transform(df2.iloc[:,2:9])

In [12]:
#Split the data into training and testing
y=df['SeriousDlqin2yrs']

X_train_s, X_test_s, y_train, y_test = train_test_split(df_scale,y ,
                                   random_state=14,
                                   test_size=0.2,
                                   shuffle=True)

#### Linear Regression

In [13]:
# Linear Regression with standardized data

l_reg = LinearRegression(copy_X=True, fit_intercept=True)
l_reg.fit(X_train_s,y_train)

# Calculate the AUC
l_reg_auc_s = roc_auc_score(y_test, l_reg.predict(X_test_s))
l_reg_auc_s

0.6852609404024366

#### Ridge Regularization

In [14]:
reg_ridge = Ridge(alpha=0.2, fit_intercept=True, copy_X=True)
reg_ridge.fit(X_train_s, y_train)

# Calculate the AUC
ridge_auc_s = roc_auc_score(y_test, reg_ridge.predict(X_test_s))
ridge_auc_s

0.6852609130934181

#### Lasso Regularization

In [15]:
reg_lasso = Lasso(alpha=0.2, fit_intercept=True, copy_X=True)
reg_lasso.fit(X_train_s, y_train)

# Calculate the AUC
lasso_auc_s = roc_auc_score(y_test, reg_lasso.predict(X_test_s))
lasso_auc_s

0.5

#### Logisitic Regression

In [16]:
#Define the model with random hyperparameters
logit = LogisticRegression(n_jobs = -1)

#Fit data into the model
logit.fit(X_train_s, y_train)

#Predict probabilities
logit_scores_proba = logit.predict_proba(X_test_s)
preds = logit_scores_proba[:,1]  # predictions

log_auc_s = round(roc_auc_score(y_test,preds),5)

#Printing area under curve
print('AUC Score : ',  log_auc_s)

AUC Score :  0.65712


#### Hyperparameter tuning

In [17]:
# Initialize logistic regression model
model = LogisticRegression(solver='liblinear')

# Define hyperparameters to tune
hyperparameters = {'C': np.logspace(-4, 4, 10), 'penalty': ['l1', 'l2'],
                   'max_iter': list(range(100,800,100))}

# Define scorer based on AUC
scorer = make_scorer(roc_auc_score, needs_proba=True)

## GRIDSEARCH

# Initialize GridSearchCV
gridsearch = GridSearchCV(model, hyperparameters, scoring=scorer, cv=5)

# Fit GridSearchCV on training data
gridsearch.fit(X_train_s, y_train)

# Get the best model
best_model = gridsearch.best_estimator_


# Predict probabilities on test data using the best model
y_pred_proba = best_model.predict_proba(X_test_s)[:, 1]

# Calculate AUC
log_grid_auc_s = round(roc_auc_score(y_test, y_pred_proba),5)

print(f'The AUC of the test set through gridsearch tuning is {log_grid_auc_s}')

## RANDOMSEARCH

# Initialize logistic regression model
model = LogisticRegression(solver='liblinear')


# Initialize RandomizedSearchCV
randomsearch = RandomizedSearchCV(model, hyperparameters, scoring=scorer, cv=5)

# Fit RandomizedSearchCV on training data
randomsearch.fit(X_train_s, y_train)

# Get the best model
best_model = randomsearch.best_estimator_


# Predict probabilities on test data using the best model
y_pred_proba = best_model.predict_proba(X_test_s)[:, 1]

# Calculate AUC
log_random_auc_s = round(roc_auc_score(y_test, y_pred_proba),5)

print(f'The AUC of the test set through randomsearch tuning is {log_random_auc_s}')


The AUC of the test set through gridsearch tuning is 0.67749
The AUC of the test set through randomsearch tuning is 0.67749


#### Random Forests

In [18]:
# Define the model
rf = RandomForestClassifier()

# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'n_jobs': [-1]
}

# Define scorer based on AUC
scorer = make_scorer(roc_auc_score, needs_proba=True)

# Initialize RandomizedSearchCV
randomsearch = RandomizedSearchCV(rf, param_grid, scoring=scorer, cv=5, verbose = 1)

# Fit RandomizedSearchCV on training data
randomsearch.fit(X_train_s, y_train)

# Get the best model
best_model = randomsearch.best_estimator_

# Get the best parameters
best_params = randomsearch.best_params_

#print(f"Best parameters: {best_params}")

# Fit the model with the best parameters
rf_best = RandomForestClassifier(**best_params)
rf_best.fit(X_train_s, y_train)

# Make predictions
y_pred = rf_best.predict(X_test_s)
y_pred_proba = rf_best.predict_proba(X_test_s)[:, 1]

# Evaluate the model
rf_auc_s = round(roc_auc_score(y_test, y_pred_proba),5)

print(f"AUC: {rf_auc_s}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
AUC: 0.8158


#### XGBoost

In [19]:
# Define the model. If GPU is present, use GPU.
model = XGBClassifier(tree_method='hist', device = 'cuda')

# Define the parameters for the RandomizedSearch
params = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1],
    'colsample_bytree': [0.6, 0.8, 1],
    'max_depth': [3, 4, 5,6]
}

#Initialize randomized search
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=5,
                                   scoring='roc_auc', n_jobs=-1, cv=5, verbose=1)

# Fit the model to the training data
random_search.fit(X_train_s, y_train)

# Make predictions
y_pred = random_search.predict(X_test_s)
y_pred_proba = random_search.predict_proba(X_test_s)[:, 1]

# Evaluate the model
xg_auc_s = roc_auc_score(y_test, y_pred_proba)

print(f"AUC: {xg_auc_s}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
AUC: 0.816369626472707


## Credibility recommendation system

In [20]:
import warnings
warnings.filterwarnings("ignore")

print("Features: 2-> RevolvingUtilizationOfUnsecuredLines, 5-> DebtRatio, 6-> MonthlyIncome, 7-> NumberOfOpenCreditLinesAndLoans, 8-> NumberOfTimes90DaysLate, 9-> NumberRealEstateLoansOrLines ")


model = XGBClassifier(tree_method='hist', device = 'cuda', subsample = 1,
 min_child_weight= 5,
 max_depth= 3,
 gamma= 2,
 colsample_bytree= 0.6 )


model.fit(X_train, y_train)
y_pred = model.predict(X_test)
thresh = 0.5
dlq_index = np.where(y_pred>thresh)

j = {}
for ind in dlq_index[0]:
  val = 0
  comb = combinations([2,5,6,7,8,9], 3)
  for i in comb:
    X_subset = df.iloc[:,[i[0],i[1],i[2]]]
    reg_subset = XGBClassifier(tree_method='hist', device = 'cuda',subsample = 1,
      min_child_weight= 5,
      max_depth= 3,
      gamma= 2,
      colsample_bytree= 0.6 )
    X_train_subset, X_test_subset, y_train_subset, y_test_subset = train_test_split(X_subset,y,
                                   random_state=12,
                                   test_size=0.2)
    reg_subset.fit(X_train_subset, y_train_subset)
    ind_pred = reg_subset.predict_proba([X_subset.loc[ind].values])[0][1]
    if ind_pred > val:
      val = ind_pred
      key = [(i[0],i[1],i[2])]
    y_pred_subset = reg_subset.predict(X_test_subset)
  j[ind] =  [val, key]
print(j)


Features: 2-> RevolvingUtilizationOfUnsecuredLines, 5-> DebtRatio, 6-> MonthlyIncome, 7-> NumberOfOpenCreditLinesAndLoans, 8-> NumberOfTimes90DaysLate, 9-> NumberRealEstateLoansOrLines 
{91: [0.10958297, [(6, 8, 9)]], 278: [0.25343886, [(5, 7, 8)]], 647: [0.052261718, [(5, 7, 8)]], 693: [0.08691117, [(6, 7, 9)]], 725: [0.06989734, [(2, 6, 9)]], 778: [0.10838944, [(6, 7, 8)]], 1005: [0.21262614, [(2, 6, 9)]], 1245: [0.08842178, [(5, 6, 8)]], 1278: [0.10179714, [(6, 7, 8)]], 1441: [0.12930675, [(5, 7, 9)]], 1529: [0.374639, [(2, 5, 8)]], 1713: [0.06408785, [(7, 8, 9)]], 1778: [0.10728434, [(5, 6, 7)]], 1924: [0.047708675, [(5, 7, 8)]], 2026: [0.13779758, [(5, 8, 9)]], 2175: [0.21848004, [(2, 6, 9)]], 2378: [0.10605118, [(6, 8, 9)]], 2399: [0.093910314, [(7, 8, 9)]], 2525: [0.10584079, [(6, 7, 9)]], 2539: [0.070598505, [(6, 7, 8)]], 3041: [0.23993474, [(2, 6, 9)]], 3097: [0.061723858, [(5, 6, 7)]], 3126: [0.08771461, [(7, 8, 9)]], 3148: [0.08548643, [(7, 8, 9)]], 3163: [0.07457859, [(7, 8